# Import Dependencies

In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
import os
import requests
import json
api_key = os.getenv("API_KEY")

# Get data

In [ ]:
url = f"https://api.oikolab.com/weather"
response = requests.get(url,
    params={'param': 'temperature',
            'location': 'Toronto, Ontario',
            'start': '1990-01-01',
            'end': '2020-12-31'},
    headers={'api-key': api_key}
    )


In [ ]:
if response.status_code == 200:
    data = response.json()
    print('Success:', response.text)
else:
    print("Error:", response.status_code, response.text)

## Save Data

In [43]:
with open('data/data.json', 'w') as f:
    json.dump(data, f)

## Load Data

In [6]:
with open('data/data.json', 'r') as f:
    js = json.load(f)
    


In [8]:
data = json.loads(js['data'])


df = pd.DataFrame(index=pd.to_datetime(data['index'], unit='s'),
                  data=data['data'],
                  columns=data['columns'])

print(df.head())



                       coordinates (lat,lon) model (name)  \
1990-01-01 00:00:00  (43.653482, -79.383935)         era5   
1990-01-01 01:00:00  (43.653482, -79.383935)         era5   
1990-01-01 02:00:00  (43.653482, -79.383935)         era5   
1990-01-01 03:00:00  (43.653482, -79.383935)         era5   
1990-01-01 04:00:00  (43.653482, -79.383935)         era5   

                     model elevation (surface)  utc_offset (hrs)  \
1990-01-01 00:00:00                     127.19              -5.0   
1990-01-01 01:00:00                     127.19              -5.0   
1990-01-01 02:00:00                     127.19              -5.0   
1990-01-01 03:00:00                     127.19              -5.0   
1990-01-01 04:00:00                     127.19              -5.0   

                     temperature (degC)  
1990-01-01 00:00:00                2.37  
1990-01-01 01:00:00                2.07  
1990-01-01 02:00:00                1.82  
1990-01-01 03:00:00                1.13  
1990-01-01 04:

# Train Model

In [16]:
# Convert the temperature to fahrenheit
df['temperature (degF)'] = df['temperature (degC)'] * 9/5 + 32



In [43]:
scaler = MinMaxScaler(feature_range=(0, 1))
temperature_scaled = scaler.fit_transform(df[['temperature (degF)']].values)
window_size = 24 * 7  # Number of past days to use for predicting the next day's temperature

# Create sequences
def create_sequences(temperatures, window_size):
    X, y = [], []
    for i in range(len(temperatures) - window_size):
        X.append(temperatures[i:i + window_size])
        y.append(temperatures[i + window_size])
    return np.array(X), np.array(y).reshape(-1, 1)


# Use the normalized temperature data for creating sequences
X, y = create_sequences(temperature_scaled, window_size)

# Split the data into training and testing sets
# It's important not to shuffle time series data to maintain the temporal sequence
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Model Creation

In [58]:
# Adjusting the model if necessary
model = Sequential([
    LSTM(100, activation='relu', input_shape=(X.shape[1], X.shape[2])),  # Increased complexity
    Dense(1)
])

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss='mean_squared_error')

In [59]:
model.fit(X_train, y_train, epochs=1, validation_split=0.2)

5432/5432 [==============================] - 1151s 212ms/step - loss: 0.0017 - val_loss: 1.6196e-04


# Model Prediction

In [61]:
latest_data = scaler.transform(df[['temperature (degF)']].values)[-window_size:].reshape((1, window_size, 1))
predicted_temperatures_normalized = []

# Recursive prediction
for _ in range(window_size):
    # Predict the next step
    next_step_normalized = model.predict(latest_data)
    
    # Store the normalized prediction
    predicted_temperatures_normalized.append(next_step_normalized[0, 0])
    
    # Update the input sequence with the new prediction
    # This moves the window one step forward by inserting the predicted value
    latest_data = np.roll(latest_data, -1, axis=1)
    latest_data[0, -1, 0] = next_step_normalized[0, 0]

# Convert normalized predictions back to the original scale (degrees Fahrenheit)
predicted_temperatures = scaler.inverse_transform(np.array(predicted_temperatures_normalized).reshape(-1, 1)).flatten()

print("Predicted temperatures for the next 7 days (168 hours):")
print(predicted_temperatures)


1/1 [==============================] - 0s 13ms/step
Predicted temperatures for the next 7 days (168 hours):
[33.830128 33.00379  32.32623  31.78369  31.380579 31.115358 30.979052
 30.956306 31.0271   31.16856  31.35685  31.568827 31.783485 31.98291
 32.15301  32.283886 32.369747 32.408726 32.40236  32.35497  32.272964
 32.164104 32.036743 31.899246 31.759409 31.62404  31.498644 31.387327
 31.292631 31.215715 31.156408 31.11343  31.084719 31.067577 31.058996
 31.055933 31.05542  31.054853 31.052011 31.045218 31.033308 31.015661
 30.992146 30.963062 30.929018 30.890905 30.849743 30.8066   30.762554
 30.718565 30.67545  30.6339   30.594358 30.557112 30.522247 30.489716
 30.459309 30.430721 30.403595 30.37754  30.35216  30.327085 30.301989
 30.276604 30.25074  30.224247 30.197102 30.169254 30.140808 30.111826
 30.082441 30.052778 30.023003 29.993221 29.963589 29.934189 29.905107
 29.876394 29.848095 29.820217 29.79273  29.765621 29.738848 29.71237
 29.686121 29.660059 29.63415  29.608332 2

In [62]:
model.save('model/tempModel')

2024-03-16 20:34:50.872465: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,1]
	 [[{{node inputs}}]]
2024-03-16 20:34:50.876265: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,1]
	 [[{{node inputs}}]]
2024-03-16 20:34:50.913531: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'lstm_12_input' with dtype float and shape [?,168,1]
	 [[{{node lstm_12_input}}]]


INFO:tensorflow:Assets written to: model/tempModel/assets


INFO:tensorflow:Assets written to: model/tempModel/assets


In [63]:
del model

In [ ]:
load_model('model/tempModel')